<a href="https://colab.research.google.com/github/PyLink88/PyLink88/blob/main/trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Recurrent-Autoencoder-auc_loss/Recurrent-Autoencoder-auc_loss')

In [ ]:
!python utils/create_config.py

Config successfully written


In [ ]:
!python utils/data_preparation.py 1 0.5 0.5 0.05

### Starting downloading ECG5000 data ###
### Download done! ###
Extracting all the files now...
Extraction done!
Saved data in numpy
Data preparation done!


In [ ]:
!python main.py configs/config_rnn_ae.json

 *************************************** 
Experiment name: rnn_ae_ECG5000_exp_0_b
 *************************************** 
RecurrentAEAgent
Checkpoints folder ./experiments/checkpoints/rnn_ae_ECG5000_exp_0_b already exists
Operation will be on *****GPU-CUDA***** 
Training a new model from scratch
Saved configuration in ./experiments/checkpoints/rnn_ae_ECG5000_exp_0_b/
Epoch-0-: 100% 12/12 [00:00<00:00, 18.50it/s]
Training loss at epoch 0 is 0.6717494825522105
Validation at epoch -0-: 100% 3/3 [00:00<00:00, 58.26it/s]
Validation loss at epoch 0 is 0.6674147645632426
Saving a best model
Epoch-1-: 100% 12/12 [00:00<00:00, 18.24it/s]
Training loss at epoch 1 is 0.669277181228002
Validation at epoch -1-: 100% 3/3 [00:00<00:00, 50.37it/s]
Validation loss at epoch 1 is 0.6654995282491049
Saving a best model
Epoch-2-:  50% 6/12 [00:00<00:00, 18.49it/s]Traceback (most recent call last):
  File "main.py", line 31, in <module>
    main()
  File "main.py", line 27, in main
    agent.run()
  File 

In [ ]:
## If you are running on Google Colab, uncomment below to install the necessary dependencies 
## before beginning the exercise.
## https://colab.research.google.com/github/ray-project/tutorial/blob/master/tune_exercises/exercise_2_optimize.ipynb#scrollTo=1-Mt9Cp7N8IZ


print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

# # A hack to force the runtime to restart, needed to include the above dependencies.
print("Done installing! Restarting via forced crash.")
import os
os._exit(0)

Setting up colab environment
     |████████████████████████████████| 49.7MB 81kB/s 
     |████████████████████████████████| 1.0MB 37.0MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 3.1MB 30.6MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 1.3MB 32.9MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 133kB 54.5MB/s 
     |████████████████████████████████| 122kB 42.8MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 296kB 46.1MB/s 
     |████████████████████████████████| 143kB 57.1MB/s 


In [1]:
# Imports
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from easydict import EasyDict


# Setting working directory
os.chdir('/content/drive/MyDrive/Recurrent-Autoencoder-auc_loss/Recurrent-Autoencoder-auc_loss')

from agents.rnn_autoencoder import RecurrentAEAgent
import torch
from torch import nn
from graphs.models.recurrent_autoencoder import RecurrentAE
from functools import partial

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [ ]:
# https://stackoverflow.com/questions/44260217/hyperparameter-optimization-for-pytorch-model
def tune_model(config):
    os.chdir('/content/drive/MyDrive/Recurrent-Autoencoder-auc_loss/Recurrent-Autoencoder-auc_loss')
  # Setting the configuration
    config_rnn_ae = {

    # Experiment information
    "exp_name": "rnn_ae_ECG5000_exp_0_b",
    "agent": "RecurrentAEAgent",

    # Architecture hyperparameters
    "rnn_type": "GRU",
    "rnn_act": "None",
    "n_layers": 1,
    "n_features": 1,

    # Optimization hyperparameters
    "learning_rate": 0.001,
    "batch_size": 128,
    "batch_size_val": 256,
    "max_epoch": 5,

    # Loss function
    'loss': 'MAEAUC',

    # AUC hyperparameters
    'lambda_auc': 0.1,
    'sampler_random_state': 88,

    # Folder where to retrieve the data and their names
    "data_folder": "./data/ECG5000/numpy/",
    "X_train": "X_train.npy",
    "y_train": "y_train.npy",
    "X_train_p": "X_train_p.npy",
    "y_train_p": "y_train_p.npy",
    "X_val": "X_val.npy",
    "y_val": "y_val.npy",
    "X_test": "X_test.npy",
    "y_test": "y_test.npy",
    "X_val_p": "X_val_p.npy",
    "y_val_p": "y_val_p.npy",

    # Training type: by now set equal to "one_class"
    "training_type": "more_class",
    "validation_type": "one_class",

    # Checkpoints
    "checkpoint_file": "checkpoint.pth.tar",
    "checkpoint_dir": "./experiments/checkpoints/",
    "load_checkpoint": False,

    # GPU settings
    "cuda": True,
    "device": "cuda",
    "gpu_device": 0,
    "seed": 58
}


    config_rnn_ae = EasyDict(config_rnn_ae)
    if config["loss_type"] == 'MAE':
      config_rnn_ae.training_type = 'one_class'
    else:
      config_rnn_ae.training_type = 'more_class'

    agent = RecurrentAEAgent(config_rnn_ae)

    # Setting the model
    agent.model = RecurrentAE(config["latent_dim"], agent.config)
    agent.model.to(agent.device)

    # Setting the loss
    agent.loss = agent.possible_loss[config["loss_type"]]
    agent.loss.to(agent.device)

    # Setting the optimizer
    agent.optimizer = torch.optim.Adam(agent.model.parameters(), lr = config["lr"])
    agent.train()
    perf = agent.best_valid
    tune.report(mean_accuracy=perf)



analysis = tune.run(tune_model, 
                    config={"latent_dim": tune.grid_search([32, 48, 64, 80]),
                                        "lr": tune.grid_search([0.001, 0.01, 0.1]),
                                        "loss_type": tune.grid_search(['MAE','MAEAUC'])}, 
                    resources_per_trial = {"cpu": 2, "gpu": 1})

In [ ]:
df = analysis.dataframe()

In [ ]:
df.logdir.iloc[0]

'/root/ray_results/tune_model_2021-05-14_09-25-44/tune_model_5bfe9_00000_0_latent_dim=32,lr=0.001_2021-05-14_09-25-45'